In [1]:
# ESG_scores_notebook.ipynb

import yfinance as yf
import pandas as pd
import requests
from tqdm.notebook import tqdm

# 1) Define the list of tickers you want ESG for
# Replace or load from CSV
tickers = ["AAPL", "MSFT", "TSLA", "GOOGL", "AMZN"]

# Optional: Fetch names using yfinance
def get_stock_names(ticker_list):
    names = {}
    for ticker in ticker_list:
        try:
            info = yf.Ticker(ticker).info
            names[ticker] = info.get("longName", info.get("shortName", ticker))
        except Exception as e:
            names[ticker] = ticker
    return names

names = get_stock_names(tickers)
print("Fetched company names:", names)

# 2) Define function to get ESG scores via Yahoo endpoint
def get_esg_scores(symbol):
    """
    Fetches ESG scores from Yahoo Finance (unofficial endpoint).
    Returns a dict with total esg, env, soc, gov scores or None if failed.
    """
    url = "https://query2.finance.yahoo.com/v1/finance/esgChart"
    params = {"symbol": symbol}
    try:
        r = requests.get(url, params=params, timeout=8)
        if not r.ok:
            return None
        data = r.json()
        # Some tickers might not have ESG data
        result = data.get("esgChart", {}).get("result")
        if not result or len(result) == 0:
            return None
        # The scores come time-series-like; take latest point
        series = result[0].get("symbolSeries", [])
        if not series:
            return None
        latest = series[-1]
        return {
            "esgScore": latest.get("esgScore"),
            "environmentScore": latest.get("environmentScore"),
            "socialScore": latest.get("socialScore"),
            "governanceScore": latest.get("governanceScore")
        }
    except Exception as e:
        return None

# 3) Loop over tickers and collect ESG
rows = []
for ticker in tqdm(tickers):
    esg = get_esg_scores(ticker)
    rows.append({
        "ticker": ticker,
        "name": names.get(ticker),
        "esgScore": esg.get("esgScore") if esg else None,
        "environmentScore": esg.get("environmentScore") if esg else None,
        "socialScore": esg.get("socialScore") if esg else None,
        "governanceScore": esg.get("governanceScore") if esg else None
    })

df = pd.DataFrame(rows)
df


Fetched company names: {'AAPL': 'Apple Inc.', 'MSFT': 'Microsoft Corporation', 'TSLA': 'Tesla, Inc.', 'GOOGL': 'Alphabet Inc.', 'AMZN': 'Amazon.com, Inc.'}


  0%|          | 0/5 [00:00<?, ?it/s]

,ticker,name,esgScore,environmentScore,socialScore,governanceScore
0,AAPL,Apple Inc.,None,None,None,None
1,MSFT,Microsoft Corporation,None,None,None,None
2,TSLA,"Tesla, Inc.",None,None,None,None
3,GOOGL,Alphabet Inc.,None,None,None,None
4,AMZN,"Amazon.com, Inc.",None,None,None,None


In [3]:
print(df)

  ticker                   name esgScore environmentScore socialScore  \
0   AAPL             Apple Inc.     None             None        None   
1   MSFT  Microsoft Corporation     None             None        None   
2   TSLA            Tesla, Inc.     None             None        None   
3  GOOGL          Alphabet Inc.     None             None        None   
4   AMZN       Amazon.com, Inc.     None             None        None   

  governanceScore  
0            None  
1            None  
2            None  
3            None  
4            None  


In [11]:
from yahooquery import Ticker
import pandas as pd

tickers = ["AAPL", "MSFT", "TSLA", "GOOGL", "AMZN"]
t = Ticker(tickers)

esg = t.esg_scores

rows = []

for ticker in tickers:
    data = esg.get(ticker)

    if isinstance(data, dict):
        rows.append({
            "ticker": ticker,
            "totalESG": data.get("totalEsg"),
            "environmentScore": data.get("environmentScore"),
            "socialScore": data.get("socialScore"),
            "governanceScore": data.get("governanceScore"),
            "ratingYear": data.get("ratingYear"),
        })
    else:
        # ESG data missing
        rows.append({
            "ticker": ticker,
            "totalESG": None,
            "environmentScore": None,
            "socialScore": None,
            "governanceScore": None,
            "ratingYear": None,
        })

df_esg = pd.DataFrame(rows)
df_esg


,ticker,totalESG,environmentScore,socialScore,governanceScore,ratingYear
0,AAPL,None,None,None,None,None
1,MSFT,None,None,None,None,None
2,TSLA,None,None,None,None,None
3,GOOGL,None,None,None,None,None
4,AMZN,None,None,None,None,None


In [13]:
from yahooquery import Ticker
import pandas as pd

tickers = ["AAPL", "MSFT", "TSLA", "GOOGL", "AMZN"]
t = Ticker(tickers)
esg = t.esg_scores

df = (
    pd.DataFrame(esg)
    .T
    .reset_index(names="ticker")
)

df


Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "/Users/stefangaman/anaconda3/lib/python3.11/site-packages/IPython/core/interactiveshell.py", line 3526, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/var/folders/jp/bd398j317bgfry9k1hrttp6h0000gn/T/ipykernel_10391/3055701052.py", line 9, in <module>
    pd.DataFrame(esg)
  File "/Users/stefangaman/anaconda3/lib/python3.11/site-packages/pandas/core/frame.py", line 709, in __init__
  File "/Users/stefangaman/anaconda3/lib/python3.11/site-packages/pandas/core/internals/construction.py", line 481, in dict_to_mgr
    arrays = [com.maybe_iterable_to_list(data[k]) for k in keys]
       ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/stefangaman/anaconda3/lib/python3.11/site-packages/pandas/core/internals/construction.py", line 115, in arrays_to_mgr
    else:
          
  File "/Users/stefangaman/anaconda3/lib/python3.11/site-packages/pandas/core/internals/construction.py", line 645, in _extract_i

In [15]:
from pprint import pprint
pprint(esg)


{'AAPL': 'No fundamentals data found for symbol: AAPL',
 'AMZN': 'No fundamentals data found for symbol: AMZN',
 'GOOGL': 'No fundamentals data found for symbol: GOOGL',
 'MSFT': 'No fundamentals data found for symbol: MSFT',
 'TSLA': 'No fundamentals data found for symbol: TSLA'}
